In [2]:
from pyspark.sql import SparkSession

In [3]:
# Initialize Spark session
spark = SparkSession.builder.appName("Shuffle Example").getOrCreate()

25/05/18 07:32:22 WARN Utils: Your hostname, codespaces-e1abf6 resolves to a loopback address: 127.0.0.1; using 10.0.0.54 instead (on interface eth0)
25/05/18 07:32:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/18 07:32:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Create a DataFrame with sample data
data = [
    (1, "Alice", 100),
    (2, "Bob", 200),
    (3, "Alice", 150),
    (4, "Bob", 250),
    (5, "Charlie", 300)
]

columns = ["id", "name", "amount"]
df = spark.createDataFrame(data, columns)

In [5]:
# Group by "name" and sum the "amount" to force a shuffle
grouped_df = df.groupBy("name").sum("amount")

In [6]:
grouped_df.explain(mode="extended")  # Use "extended" to get detailed plan

== Parsed Logical Plan ==
'Aggregate ['name], ['name, sum(amount#2L) AS sum(amount)#10L]
+- LogicalRDD [id#0L, name#1, amount#2L], false

== Analyzed Logical Plan ==
name: string, sum(amount): bigint
Aggregate [name#1], [name#1, sum(amount#2L) AS sum(amount)#10L]
+- LogicalRDD [id#0L, name#1, amount#2L], false

== Optimized Logical Plan ==
Aggregate [name#1], [name#1, sum(amount#2L) AS sum(amount)#10L]
+- Project [name#1, amount#2L]
   +- LogicalRDD [id#0L, name#1, amount#2L], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[name#1], functions=[sum(amount#2L)], output=[name#1, sum(amount)#10L])
   +- Exchange hashpartitioning(name#1, 200), ENSURE_REQUIREMENTS, [plan_id=15]
      +- HashAggregate(keys=[name#1], functions=[partial_sum(amount#2L)], output=[name#1, sum#14L])
         +- Project [name#1, amount#2L]
            +- Scan ExistingRDD[id#0L,name#1,amount#2L]



Query plans are read from bottom to top,indicating the flow of data through different operations.Indentation represents parallel operations,meaning those steps can occur simultaneously.

In this line, Scan ExistingRDD,data is being loaded into Spark from an in-memory dataset.This is the starting point of the execution.In this line, only name and amount columns are selected,reducing unnecessary data movement.

HashAggregate.This is the partial aggregation done before shuffle.Spark performs partial aggregation per partition.Instead of sending all records for a name,it partially sums value within each partition,reducing shuffle size.

Exchange hashpartitioning.This is the shuffle stage.Spark redistributes data across 200 partitionsusing hash partitioning on name.Now all records with the same name are in the partition.

final aggregation at HashAggregate.Spark performs the final aggregationon amount after the shuffle.

In [7]:
# Trigger execution (e.g., show results)
grouped_df.show()

25/05/18 07:32:40 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+-------+-----------+
|   name|sum(amount)|
+-------+-----------+
|    Bob|        450|
|  Alice|        250|
|Charlie|        300|
+-------+-----------+



In [8]:
#Stop Spark Session
spark.stop()